In [1]:
import pandas as pd
import gurobipy as gb
from gurobipy import *

In [2]:
corn_df = pd.read_excel('final_constraints.xlsx',sheet_name='Corn')
corn_df

,herbicide_name,nebraska_yield_coef,illinois_yield_coef,iowa_yield_coef,lb,ub,w0,w1,w2,w3,...,w13,w14,w15,w16,w17,w18,w19,w20,summing_constraint,binary
0,"2,4-D",3.500000,7.250000,8.750000,NaN,NaN,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.000,NaN
1,ALACHLOR,1.500000,7.250000,14.000000,2.500,3.0000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000,NaN
2,ATRAZINE,2.833333,7.250000,5.666667,1.600,2.0000,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
3,BROMOXYNIL,2.833333,6.000000,17.500000,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
4,DICAMBA,3.166667,2.000000,8.750000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,DIFLUFENZOPYR,0.000000,2.000000,12.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sum W16 to W21 == 0
6,EPTC,3.166667,6.000000,9.333333,3.140,6.1380,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,FORAMSULFURON,1.000000,1.500000,0.000000,NaN,NaN,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.041,sum W8 to W20 == 0
8,GLYPHOSATE,3.166667,7.250000,8.750000,1.375,4.5375,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,HALOSULFURON,2.166667,7.250000,8.750000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.076,sum all W == 2


In [3]:
coef =[corn_df['nebraska_yield_coef'].to_list()]
coef.append(corn_df['illinois_yield_coef'].to_list())
coef.append(corn_df['iowa_yield_coef'].values.tolist())

In [4]:
coef

[[3.49999999999999,
  1.5,
  2.83333333333333,
  2.83333333333333,
  3.16666666666666,
  0.0,
  3.16666666666666,
  1.0,
  3.16666666666666,
  2.16666666666666,
  2.5,
  2.25,
  2.25],
 [7.25,
  7.25,
  7.25000000000001,
  6.00000000000002,
  2.0,
  2.00000000000001,
  6.00000000000001,
  1.50000000000001,
  7.25000000000001,
  7.25,
  13.3333333333333,
  5.00000000000001,
  0.0],
 [8.74999999999999,
  13.9999999999999,
  5.66666666666666,
  17.5,
  8.74999999999999,
  11.9999999999999,
  9.33333333333333,
  0.0,
  8.74999999999999,
  8.74999999999999,
  5.66666666666667,
  8.74999999999998,
  5.0]]

In [5]:
corn_restrictions = pd.read_csv('corn_herbicide_restriction.csv')
corn_restrictions.head()

,name,lower,upper,type
0,ALACHLOR,2.50000,3.0000,before
1,ATRAZINE,1.60000,2.0000,before
2,EPTC,3.14000,6.1380,before
3,GLYPHOSATE,1.37500,4.5375,before
4,S-METOLACHLOR,1.42875,2.3800,before


In [6]:
restrictions = pd.read_csv('corn_herbicide_restriction_2.csv')
# restrictions = pd.read_excel('corn_herbicide_restriction.xlsx',sheet_name='new')
restrictions.rename(columns={'Unnamed: 0':'herbicide'},inplace=True)
restrictions

,herbicide,before_lb,before_ub,before,during_lb,during_ub,during,after_lb,after_ub,after
0,"2,4-D",0.000000,0.000000,0,0.000000,0.000000,0,0.107728,0.215456,1
1,ALACHLOR,1.133980,1.360776,1,0.514364,0.617237,1,0.000000,0.000000,0
2,ATRAZINE,0.725747,0.907184,1,0.329193,0.411491,1,0.725747,0.907184,1
3,BROMOXYNIL,0.000000,0.000000,0,0.000000,0.000000,0,0.056699,0.113398,1
4,DICAMBA,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0
5,DIFLUFENZOPYR,0.000000,0.000000,0,0.000000,0.000000,0,0.079379,0.119068,1
6,EPTC,1.424279,2.784148,1,0.646042,0.818456,1,0.000000,0.000000,0
7,FORAMSULFURON,0.000000,0.000000,0,0.000000,0.000000,0,0.014882,0.014882,1
8,GLYPHOSATE,0.623689,2.058174,1,0.282900,0.933571,1,0.000000,0.000000,0
9,HALOSULFURON,0.000000,0.000000,0,0.102873,0.205746,1,0.014016,0.028259,1


In [18]:
corn_state_name = ['Nebraska','Illinois','Iowa']
baseline = [178.5, 178, 184.9]
yield_minimum = [181, 201, 202] # yield >= baseline + each herbicde * coef
num_weeks = 28
corn_herbicdes = corn_df['herbicide_name'].unique()
corn_before_lb = corn_restrictions[corn_restrictions['type'] == 'before'][['name','lower']]
corn_before_ub = corn_restrictions[corn_restrictions['type'] == 'before'][['name','upper']]



num_herbicides_corn = len(corn_herbicdes)
Range_herbicides = range(num_herbicides_corn)
Range_weeks = range(num_weeks)
Range_preplant = range(0,4)
Range_atplant = range(4,19)
Range_postplant = range(19,28)


model = gb.Model('Herbicide Minimization for Corn')

NE = model.addVars(num_herbicides_corn, num_weeks, vtype=gb.GRB.CONTINUOUS, name=('{} quantity used for week {} in Nebraska'.format(i,j) for i in corn_herbicdes for j in range(1,num_weeks+1)))
# IL = model.addVars(num_herbicides_corn, num_weeks, vtype=gb.GRB.CONTINUOUS, name=('{} quantity used for week {} in Illinois'.format(i,j) for i in corn_herbicdes for j in range(1,num_weeks+1)))
# IO = model.addVars(num_herbicides_corn, num_weeks, vtype=gb.GRB.CONTINUOUS, name=('{} quantity used for week {} in Iowa'.format(i,j) for i in corn_herbicdes for j in range(1,num_weeks+1)))

NE_z = model.addVars(num_herbicides_corn, num_weeks, vtype=gb.GRB.BINARY, name=('prescence of {} for week {} in Nebraska'.format(i,j) for i in corn_herbicdes for j in range(1,num_weeks+1)))
# IL_z = model.addVars(num_herbicides_corn, num_weeks, vtype=gb.GRB.BINARY, name=('prescence of {} for week {} in Illinois'.format(i,j) for i in corn_herbicdes for j in range(1,num_weeks+1)))
# IO_z = model.addVars(num_herbicides_corn, num_weeks, vtype=gb.GRB.BINARY, name=('prescence of {} for week {} in Iowa'.format(i,j) for i in corn_herbicdes for j in range(1,num_weeks+1)))


#nebraska= gb.quicksum(NE[i,j]*NE_z[i,j] for i in range(num_herbicides_corn) for j in range(0,num_weeks))
# illinois = gb.quicksum(IL[i,j]*IL_z[i,j] for i in range(num_herbicides_corn) for j in range(0,num_weeks))
# iowa = gb.quicksum(IO[i,j]*IO_z[i,j] for i in range(num_herbicides_corn) for j in range(0,num_weeks))
#total_usage = nebraska #+ illinois + iowa

model.ModelSense = gb.GRB.MINIMIZE
nebraska=model.addVar(vtype=gb.GRB.CONTINUOUS, name='nebraska')

model.addConstr(nebraska== gb.quicksum(NE[i,j]*NE_z[i,j] for i in range(num_herbicides_corn) for j in range(0,num_weeks)))
model.setObjectiveN(nebraska,priority=0,index=1)
glyphosate=model.addVar(vtype=gb.GRB.CONTINUOUS, name='glyphosate')
model.addConstr(glyphosate== gb.quicksum(NE[8,j]*NE_z[8,j] for j in Range_weeks))
model.setObjectiveN(glyphosate, priority=1,index=0)
#model.setObjective(glyphosate,gb.GRB.MINIMIZE)

# Constraints
# 1. Each state has a minimum amount of herbicide used for ensuring production yield
#model.addConstr(baseline[0]+quicksum(quicksum(NE[i,j] for j in range(0,num_weeks))*coef[0][i] for i in Range_herbicides) >= baseline[0], name='NE minimum yield')
# model.addConstr(baseline[0]+quicksum(quicksum(NE[i,j] for j in range(0,num_weeks))*coef[0][i] for i in Range_herbicides) >= yield_minimum[0], name='NE minimum yield')
# model.addConstr(baseline[1]+quicksum(quicksum(IL[i,j] for j in range(0,num_weeks))*coef[1][i] for i in Range_herbicides) >= yield_minimum[1], name='IL minimum yield')
# model.addConstr(baseline[2]+quicksum(quicksum(IO[i,j] for j in range(0,num_weeks))*coef[2][i] for i in Range_herbicides) >= yield_minimum[2], name='IO minimum yield')


herbicide_names = restrictions['herbicide'].values

# 2. Each state has a lower and upper bound of herbicide used for beofre planting
# for i in range(herbicide_names.shape[0]):
#     herbicide = herbicide_names[i]
#     row = restrictions[restrictions['herbicide'] == herbicide]
#     model.addConstrs(NE[i,j]*NE_z[i,j] *row['before'].values[0] <= row['before_ub'].values[0] for j in Range_preplant)
#     print(herbicide)
#     print(row['before_lb'].values[0]*row['before'].values[0])
#     model.addConstrs(NE[i,j]*NE_z[i,j] >= row['before_lb'].values[0]*row['before'].values[0] for j in Range_preplant)
    
    # model.addConstr(quicksum(IL[i,j] for j in Range_preplant)*row['before'].values[0] <= row['before_ub'].values[0])
    # model.addConstr(quicksum(IL[i,j] for j in Range_preplant) >= row['before_lb'].values[0]*row['before'].values[0])

    # model.addConstr(quicksum(IO[i,j] for j in Range_preplant)*row['before'].values[0] <= row['before_ub'].values[0])
    # model.addConstr(quicksum(IO[i,j] for j in Range_preplant) >= row['before_lb'].values[0]*row['before'].values[0])

# 3. Each state has a lower and upper bound of herbicide used for at planting
# for i in range(herbicide_names.shape[0]):
#     herbicide = herbicide_names[i]
#     row = restrictions[restrictions['herbicide'] == herbicide]
#     model.addConstr(quicksum(NE[i,j] for j in Range_atplant)*row['during'].values[0] <= row['during_ub'].values[0])
#     model.addConstr(quicksum(NE[i,j] for j in Range_atplant) >= row['during_lb'].values[0]*row['during'].values[0])

    # model.addConstr(quicksum(IL[i,j] for j in Range_atplant)*row['during'].values[0] <= row['during_ub'].values[0])
    # model.addConstr(quicksum(IL[i,j] for j in Range_atplant) >= row['during_lb'].values[0]*row['during'].values[0])

    # model.addConstr(quicksum(IO[i,j] for j in Range_atplant)*row['during'].values[0] <= row['during_ub'].values[0])
    # model.addConstr(quicksum(IO[i,j] for j in Range_atplant) >= row['during_lb'].values[0]*row['during'].values[0])

# 4. Each state has a lower and upper bound of herbicide used for post planting
# for i in range(herbicide_names.shape[0]):
#     herbicide = herbicide_names[i]
#     row = restrictions[restrictions['herbicide'] == herbicide]
#     model.addConstr(quicksum(NE[i,j] for j in Range_postplant)*row['after'].values[0] <= row['after_ub'].values[0])
#     model.addConstr(quicksum(NE[i,j] for j in Range_postplant) >= row['after_lb'].values[0]*row['after'].values[0])
    
    # model.addConstr(quicksum(IL[i,j] for j in Range_postplant)*row['after'].values[0] <= row['after_ub'].values[0])
    # model.addConstr(quicksum(IL[i,j] for j in Range_postplant) >= row['after_lb'].values[0]*row['after'].values[0])

    # model.addConstr(quicksum(IO[i,j] for j in Range_postplant)*row['after'].values[0] <= row['after_ub'].values[0])
    # model.addConstr(quicksum(IO[i,j] for j in Range_postplant) >= row['after_lb'].values[0]*row['after'].values[0])

# 5. If a herbicide is used in a stage, the slack variable is greater than 1
for i in Range_herbicides:

    row = restrictions[restrictions['herbicide'] == herbicide_names[i]]
    for j in Range_preplant:
        model.addConstr(quicksum(NE_z[i,j] for j in Range_preplant) >= row['before'].values[0])
        # model.addConstr(quicksum(IL_z[i,j] for j in Range_preplant) >= row['before'].values[0])
        # model.addConstr(quicksum(IO_z[i,j] for j in Range_preplant) >= row['before'].values[0])

    for j in Range_atplant:
        # row = restrictions[restrictions['herbicide'] == herbicide]
        model.addConstr(quicksum(NE_z[i,j] for j in Range_atplant) >= row['during'].values[0])
        # model.addConstr(quicksum(IL_z[i,j] for j in Range_atplant) >= row['during'].values[0])
        # model.addConstr(quicksum(IO_z[i,j] for j in Range_atplant) >= row['during'].values[0])

    for j in Range_postplant:
        # row = restrictions[restrictions['herbicide'] == herbicide]
        model.addConstr(quicksum(NE_z[i,j] for j in Range_postplant) >= row['after'].values[0])
        # model.addConstr(quicksum(IL_z[i,j] for j in Range_postplant) >= row['after'].values[0])
        # model.addConstr(quicksum(IO_z[i,j] for j in Range_postplant) >= row['after'].values[0])

# 6. Special Quantity Restrictions
    # 2,4-D W0-W18 at <= 0.41 kg/acre (0.855pt/acre) (1 L/ha)
    model.addConstr(quicksum(NE[0,j]*NE_z[0,j] for j in range(18))<=0.41)
    # model.addConstr(quicksum(IL[0,j]*IL_z[0,j] for j in range(18))<=0.41)
    # model.addConstr(quicksum(IO[0,j]*IO_z[0,j] for j in range(18))<=0.41)

    # alachlor W4-W8 at 2.05 kg/acre (4.28 pt/acre) (5 L/ha)
    model.addConstr(quicksum(NE[1,j]*NE_z[1,j] for j in range(3,8))<=2.05)
    # model.addConstr(quicksum(IL[1,j]*IL_z[1,j] for j in range(3,8))<=2.05)
    # model.addConstr(quicksum(IO[1,j]*IO_z[1,j] for j in range(3,8))<=2.05)

    # atrazine W0-W3 <= 1.814368kg/acre/year, W4-W8 at <= 1.91kg/acre (4 pt/acre/year)
    model.addConstr(quicksum(NE[2,j]*NE_z[2,j] for j in range(4))<=1.814368)
    model.addConstr(quicksum(NE[2,j]*NE_z[2,j] for j in range(3,8))<=1.91)
    # model.addConstr(quicksum(IL[2,j]*IL_z[2,j] for j in range(4))<=1.814368)
    # model.addConstr(quicksum(IL[2,j]*IL_z[2,j] for j in range(3,8))<=1.91)
    # model.addConstr(quicksum(IO[2,j]*IO_z[2,j] for j in range(4))<=1.814368)
    # model.addConstr(quicksum(IO[2,j]*IO_z[2,j] for j in range(3,8))<=1.91)

    # bromoxynil from W8 - W17 0.4 - 0.49 kg/acre
    model.addConstr(quicksum(NE[3,j]*NE_z[3,j] for j in range(7,17))<=0.49)
    model.addConstr(quicksum(NE[3,j]*NE_z[3,j] for j in range(7,17))>=0.4)
    # model.addConstr(quicksum(IL[3,j]*IL_z[3,j] for j in range(7,17))<=0.49)
    # model.addConstr(quicksum(IL[3,j]*IL_z[3,j] for j in range(7,17))>=0.4)
    # model.addConstr(quicksum(IO[3,j]*IO_z[3,j] for j in range(7,17))<=0.49)
    # model.addConstr(quicksum(IO[3,j]*IO_z[3,j] for j in range(7,17))>=0.4)

    # dicamba W4-W6 at 0.506kg/acre (1.25 L/ha)
    model.addConstr(quicksum(NE[4,j]*NE_z[4,j] for j in range(3,6))==0.506)
    # model.addConstr(quicksum(IL[4,j]*IL_z[4,j] for j in range(3,6))==0.506)
    # model.addConstr(quicksum(IO[4,j]*IO_z[4,j] for j in range(3,6))==0.506)

    # diflufenzoapyr before 2W at <= 0.17kg/acre (6 ozs/acre)
    model.addConstr(quicksum(NE[5,j]*NE_z[5,j] for j in range(2))<=0.17)
    # model.addConstr(quicksum(IL[5,j]*IL_z[5,j] for j in range(2))<=0.17)
    # model.addConstr(quicksum(IO[5,j]*IO_z[5,j] for j in range(2))<=0.17)

    # eptc W0-W3 <=2.784kg/acre (7.33 pt/acre)
    model.addConstr(quicksum(NE[6,j]*NE_z[6,j] for j in range(4))<=2.784)
    # model.addConstr(quicksum(IL[6,j]*IL_z[6,j] for j in range(4))<=2.784)
    # model.addConstr(quicksum(IO[6,j]*IO_z[6,j] for j in range(4))<=2.784)

    # foramsulfuron before W8 <= 0.04082 kg/acre per year
    model.addConstr(quicksum(NE[7,j]*NE_z[7,j] for j in range(7))<=0.04082)
    # model.addConstr(quicksum(IL[7,j]*IL_z[7,j] for j in range(7))<=0.04082)
    # model.addConstr(quicksum(IO[7,j]*IO_z[7,j] for j in range(7))<=0.04082)

    # glyphosate last application <W19 <=2.0581737kg/acre/year
    model.addConstr(quicksum(NE[8,j]*NE_z[8,j] for j in range(18,28))<=2.0581737)
    # model.addConstr(quicksum(IL[8,j]*IL_z[8,j] for j in range(18,28))<=2.0581737)
    # model.addConstr(quicksum(IO[8,j]*IO_z[8,j] for j in range(18,28))<=2.0581737)

    # halosulfron <=0.07541 kg/acre/year <=2 application/year, do not exceed per time
    #######################TO DO: PER TIME###################################
    model.addConstr(quicksum(NE[9,j]*NE_z[9,j] for j in range(28))<=0.07541*2)
    # model.addConstr(quicksum(IL[9,j]*IL_z[9,j] for j in range(28))<=0.07541*2)
    # model.addConstr(quicksum(IO[9,j]*IO_z[9,j] for j in range(28))<=0.07541*2)

    # nicosulfron <=0.0709764kg/acre
    model.addConstr(quicksum(NE[10,j]*NE_z[10,j] for j in range(28))<=0.0709764)
    # model.addConstr(quicksum(IL[10,j]*IL_z[10,j] for j in range(28))<=0.0709764)
    # model.addConstr(quicksum(IO[10,j]*IO_z[10,j] for j in range(28))<=0.0709764)

    #paraquat none

    # pendimethalin none

# 7. Presence restriction
    # 2,4-D 1 treatment per year
    model.addConstr(quicksum(NE_z[0,j] for j in range(28))<=1)
    # model.addConstr(quicksum(IL_z[0,j] for j in range(28))<=1)
    # model.addConstr(quicksum(IO_z[0,j] for j in range(28))<=1)

    # alachlor from W14 stop using
    model.addConstr(quicksum(NE_z[1,j] for j in range(13,28))==0)
    # model.addConstr(quicksum(IL_z[1,j] for j in range(13,28))==0)
    # model.addConstr(quicksum(IO_z[1,j] for j in range(13,28))==0)

    # atrazine none


    # bromoxynil Minimum re-treatment interval for the second application is 21 days
    bromoxynil_treat1 = model.addVar(vtype=GRB.INTEGER, lb=0, ub=24, name="week of the first treatment of bromoxynil")
    bromoxynil_treat2 = model.addVar(vtype=GRB.INTEGER, lb=3, ub=27, name="week of the second treatment of bromoxynil")
    model.addConstr(bromoxynil_treat1 <= 3 + bromoxynil_treat2)
    # model.addConstr(bromoxynil_treat1 == quicksum(NE_z[3,j]*Range_weeks[j] for j in Range_weeks))
    model.addConstr(quicksum(NE_z[3,j] for j in Range_weeks)==2)
    # model.addConstr(quicksum(IL_z[3,j] for j in Range_weeks)==2)
    # model.addConstr(quicksum(IO_z[3,j] for j in Range_weeks)==2)

    # dicamba from W16 do not use
    model.addConstr(quicksum(NE_z[4,j] for j in range(15,28))==0)

    # diflufenzoapyr from W16 do not use
    # model.addConstr(quicksum(NE_z[5,j] for j in range(15,28))==0)

    # foramsulfuron from W9 do not use
    # model.addConstr(quicksum(NE_z[7,j] for j in range(8,28))==0)

    # glyphosate from W19 do not use
    model.addConstr(quicksum(NE_z[8,j] for j in range(18,28))==0)

    # halosulfuron <=2 applications per year
    model.addConstr(quicksum(NE_z[9,j] for j in range(28))<=2)

    # nicosulfuron none

    # paraquat <=3 applications per year, not use between W4 and W8
    model.addConstr(quicksum(NE_z[11,j] for j in range(28))<=3)
    model.addConstr(quicksum(NE_z[11,j] for j in range(3,8))==0)

    # pendimethalin <=1 applications per year
    model.addConstr(quicksum(NE_z[12,j] for j in range(28))<=1)



    # 8. Mixing restriction
    # 2,4-D do not mix with glyphosate
    model.addConstrs(NE_z[0,j] + NE_z[8,j]  <= 1 for j in range(28))

    # parquat do not mix with glyphosate
    model.addConstrs(NE_z[11,j] + NE_z[8,j]  <= 1 for j in range(28))








M = 1e7
eps=0.00001
#force y=1 when x>0; x<=0, y could be anything
model.addConstrs(NE[i,j] <= NE_z[i,j]*M for i in Range_herbicides for j in Range_weeks)
#force y=0 when x<=0
model.addConstrs(NE[i,j] >= NE_z[i,j]*eps for i in Range_herbicides for j in Range_weeks)
# model.addConstrs(IL[i,j] <= IL_z[i,j]*M for i in Range_herbicides for j in Range_weeks)
# model.addConstrs(IO[i,j] <= IO_z[i,j]*M for i in Range_herbicides for j in Range_weeks)


model.optimize()

print('Obj: %g' % model.objVal)
for v in model.getVars():
    if v.x > 0:
        print('%s %g' % (v.varName, v.x))
for i in Range_herbicides:
    print(herbicide_names[i], "used for pre-planting",quicksum(NE[i,j].x for j in Range_preplant))
    print(herbicide_names[i], "used for at planting",quicksum(NE[i,j].x for j in Range_atplant))
    print(herbicide_names[i], "used for post-planting",quicksum(NE[i,j].x for j in Range_postplant))
          
print("Yield for each state: ")
print("Nebraska: ", baseline[0]+ quicksum(NE[i,j].x*coef[0][i] for i in Range_herbicides for j in Range_weeks))
# print("Illinois: ", baseline[1]+illinois.getValue())
# print("Iowa: ", baseline[2]+iowa.getValue())
for i in Range_herbicides:
    print(herbicide_names[i], quicksum(NE[i,j].x*NE_z[i,j].x for j in Range_weeks))
    print(herbicide_names[i], quicksum(NE_z[i,j].x for j in Range_weeks))

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1950 rows, 756 columns and 9503 nonzeros
Model fingerprint: 0x34a641ae
Model has 171 quadratic constraints
Variable types: 366 continuous, 390 integer (364 binary)
Coefficient statistics:
  Matrix range     [1e-05, 1e+07]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 3e+00]
  QRHS range       [4e-02, 3e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 1336 rows and 162 columns
Presolv

In [73]:
corn_before_lb

,name,lower
0,ALACHLOR,2.50000
1,ATRAZINE,1.60000
2,EPTC,3.14000
3,GLYPHOSATE,1.37500
4,S-METOLACHLOR,1.42875
5,PARAQUAT,0.50000
